# Sunrise Housing Case study

## Objective

Identify prospective properties to invest in based on the prediction of the actual value of these properties.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('ggplot')

In [2]:
# load the dataset
inp0 = pd.read_csv('train.csv', header=0)
inp0.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### I - Data understanding

**1. Number of rows, columns and column names**

In [3]:
inp0.shape

(1460, 81)

In [4]:
print(list(inp0.columns))

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

**Observation**<br>
1. Number of rows = 1460. Number of columns = 81.<br>
2. Large number of columns implies that many will need to be dropped or they can be used to derive new metrics.

**2. Drop columns based on definition in data dictionary**

In [5]:
#columns contain irrelevant information based on repetition, uniqueness, data dictionary description
col_drop = ['Id','Condition1','Condition2','BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2','BsmtUnfSF','MiscVal']

**3. Presence of NaN or Null**

In [6]:
#presence of NaN values & Null values
print('NaN : ' , list( [inp0[i].isna().sum() for i in inp0.columns]))
print('Null: ', list( [inp0[i].isnull().sum() for i in inp0.columns]))

NaN :  [0, 0, 0, 259, 0, 0, 1369, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 37, 37, 38, 37, 0, 38, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 690, 81, 81, 81, 0, 0, 81, 81, 0, 0, 0, 0, 0, 0, 0, 1453, 1179, 1406, 0, 0, 0, 0, 0, 0]
Null:  [0, 0, 0, 259, 0, 0, 1369, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 37, 37, 38, 37, 0, 38, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 690, 81, 81, 81, 0, 0, 81, 81, 0, 0, 0, 0, 0, 0, 0, 1453, 1179, 1406, 0, 0, 0, 0, 0, 0]


**Observation**<br>
The number of NaN and Null values are same for all the columns in the dataset. <br>
**Action**<br>
Find columns and the percentage of the NaN values which are non-zero to determine further action.

In [7]:
#many NaN & Null values present; find the columns names for more information and store in a dictionary with percentages
inp0_dict0 = {str(i) : round(((inp0[i].isna().sum()/inp0.shape[0])* 100),2) for i in inp0.columns if inp0[i].isna().sum()>0 }

#sort the dictionary with highest percentages first and print; returns a tuple
inp0_dict1 = sorted( inp0_dict0.items() , key = lambda x : x[1], reverse=True)

#convert tuple to dict
inp0_dict0 = { }
inp0_dict0 = { key:value for key,value in inp0_dict1 }

#print columns with NaN values
print(inp0_dict0)

{'PoolQC': 99.52, 'MiscFeature': 96.3, 'Alley': 93.77, 'Fence': 80.75, 'FireplaceQu': 47.26, 'LotFrontage': 17.74, 'GarageType': 5.55, 'GarageYrBlt': 5.55, 'GarageFinish': 5.55, 'GarageQual': 5.55, 'GarageCond': 5.55, 'BsmtExposure': 2.6, 'BsmtFinType2': 2.6, 'BsmtQual': 2.53, 'BsmtCond': 2.53, 'BsmtFinType1': 2.53, 'MasVnrType': 0.55, 'MasVnrArea': 0.55, 'Electrical': 0.07}


**Observation**<br>
Drop the columns with very high percentage of NaN values ie above 30%. These columns have NA values meaning None for categorical variables implying the absence of the said feature which is very high. For remaining columns, find the common number of rows satisfying them and drop them if required else, impute these columns.

In [8]:
#dropping columns with missing values above 30%
col_drop.extend([key for key,value in inp0_dict0.items() if value >= 30])
inp0.drop(col_drop, axis=1, inplace=True)

**4. Determine columns with high number of zeros and to add them to drop list**

In [9]:
# 50% of number of rows
.3*(inp0.shape[0])

438.0

In [10]:
#find all columns with high number of zeroes and drop them
inp0_0col = [i for i in inp0.columns if (inp0[i] == 0).sum()>438]
col_drop=[]
col_drop.extend(inp0_0col)
inp0.drop(col_drop, axis=1, inplace=True)

In [11]:
print(list(inp0.columns))

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice']


In [12]:
print([ (inp0[i] == 0).sum() for i in inp0.columns ])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 0, 0, 9, 6, 1, 0, 0, 0, 0, 0, 0, 81, 81, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
print(len(list(inp0.columns)))

54
